In [ ]:
import pandas as pd
import json
import math
#ENTER 0 if the difficulty is the lowest diff, 1 if it is second lowest, etc
diffNum = 0 # change this number

In [ ]:
def get_pythagoras(x, y):
    return math.sqrt(x ** 2 + y ** 2)
diffPath = 'accsaber-maps/362bb (Novocaine - Kassi)/ExpertPlusStandard.dat' #Replace with your difficulty file's path
infoPath = 'accsaber-maps/362bb (Novocaine - Kassi)/Info.dat' #Replace with your info file's path
with open(diffPath) as diff_json_data:
    diffData = json.load(diff_json_data)

with open(infoPath) as info_json_data:
    infoData = json.load(info_json_data)

In [ ]:
#Parse BPM changes if they exist
initialBPM = infoData.get('_beatsPerMinute')
diffDict = infoData.get('_difficultyBeatmapSets')[0].get('_difficultyBeatmaps')[diffNum]
njs = diffDict.get('_noteJumpMovementSpeed')
bpmChangesDict = diffData.get('_customData').get('_BPMChanges')
df_BPMChanges = pd.DataFrame(bpmChangesDict)

In [ ]:
df = pd.DataFrame(diffData['_notes'])
df['_yCenter'] = df.loc[:, ('_lineLayer')].apply(lambda x: 1 + x * 0.55)
df['_xCenter'] = df.loc[:, ('_lineIndex')].apply(lambda x: -0.9 + x * 0.6)

#Add bpm column
df['_bpm'] = initialBPM
for i in range(len(df)):
    currentTime = df.loc[i, '_time']
    currentRow = 0
    for j in range(len(df_BPMChanges)):
        if currentTime >= df_BPMChanges.loc[j, '_time']:
            df['_bpm'] = df_BPMChanges.loc[j, '_BPM']
   

        
left = (df[df['_type'] == 0]) #All left handed notes

right = (df[df['_type'] == 1]) #All right handed notes


left['_timeChange'] = left.loc[:, ['_time']].diff().fillna(0)
right['_timeChange'] = right.loc[:, ['_time']].diff().fillna(0)


left['_timeChangeSeconds'] = (60 * left['_timeChange']) / left['_bpm']
right['_timeChangeSeconds'] = (60 * right['_timeChange']) / right['_bpm']

In [ ]:
#Account for sliders and stacks
EPSILON = 0.059

df_newLeftSwing = left[(left['_timeChange'] > EPSILON) | (left.index == 0)]
df_newRightSwing = right[(right['_timeChange'] > EPSILON) | (right.index == 0)]

df_newLeftSwing['_xMovement'] = df_newLeftSwing.loc[:, ['_xCenter']].diff().fillna(0)
df_newLeftSwing['_yMovement'] = df_newLeftSwing.loc[:, ['_yCenter']].diff().fillna(0)
df_newLeftSwing['_totMovement'] = df_newLeftSwing.apply(lambda x: get_pythagoras(x['_xMovement'], x['_yMovement']), axis=1).fillna(0) 
df_newLeftSwing['_angleMagnitudeChange'] = abs(df_newLeftSwing.apply(lambda x: math.atan(x['_yMovement']/x['_xMovement']), axis=1))
df_newLeftSwing['_timeChange'] = df_newLeftSwing.loc[:, ['_time']].diff().fillna(0)
df_newLeftSwing['_timeChangeSeconds'] = (60 * df_newLeftSwing['_timeChange']) / df_newLeftSwing['_bpm']

df_newRightSwing['_xMovement'] = df_newRightSwing.loc[:, ['_xCenter']].diff().fillna(0)
df_newRightSwing['_yMovement'] = df_newRightSwing.loc[:, ['_yCenter']].diff().fillna(0)
df_newRightSwing['_totMovement'] = df_newRightSwing.apply(lambda x: get_pythagoras(x['_xMovement'], x['_yMovement']), axis=1).fillna(0)
df_newRightSwing['_angleMagnitudeChange'] = abs(df_newRightSwing.apply(lambda x: math.atan(x['_yMovement']/x['_xMovement']), axis=1))
df_newRightSwing['_timeChange'] = df_newRightSwing.loc[:, ['_time']].diff().fillna(0)
df_newRightSwing['_timeChangeSeconds'] = (60 * df_newRightSwing['_timeChange']) / df_newRightSwing['_bpm']

df_newSwing = pd.concat([df_newLeftSwing, df_newRightSwing])

df_newSwing = df_newSwing.sort_values('_time')
df_newSwing['_timeChange'] = df_newSwing.loc[:, ['_time']].diff().fillna(0)
df_newSwing['_timeChangeSeconds'] = (60 * df_newSwing['_timeChange']) / df_newSwing['_bpm']


df_newLeftSwing['_seconds'] = df_newLeftSwing['_timeChangeSeconds'].cumsum()
df_newRightSwing['_seconds'] = df_newRightSwing['_timeChangeSeconds'].cumsum()
df_newSwing['_seconds'] = df_newSwing['_timeChangeSeconds'].cumsum()


#Statistics 

left_swings = len(df_newLeftSwing)
right_swings = len(df_newRightSwing)
total_swings = len(df_newSwing)
left_time = left['_timeChangeSeconds'].sum()
right_time = right['_timeChangeSeconds'].sum()
total_time = left_time + right_time
right_avg_sps = right_swings / right_time
left_avg_sps = left_swings / left_time
avg_sps = total_swings / total_time
left_avg_angleChange = df_newLeftSwing['_angleMagnitudeChange'].mean()
right_avg_angleChange = df_newRightSwing['_angleMagnitudeChange'].mean()
avg_angleChange = df_newSwing['_angleMagnitudeChange'].mean()

In [ ]:
#The code in this cell is adapted from Uninstaller's sps calculator


def calculate_swings_list(df_current):
    swing_list = df_current['_seconds'].tolist()
    last = math.floor(df_current['_seconds'].max())
    array = [0 for x in range(math.floor(last) + 1)]
    for swing in swing_list:
        array[math.floor(swing)] += 1
    return array

    

def calculate_max_sps(swings_list, interval = 10):
    
    current_sps_sum = sum(swings_list[:interval])
    max_sps_sum = current_sps_sum
    for x in range(0, len(swings_list) - interval):
        current_sps_sum = current_sps_sum - swings_list[x] + swings_list[x + interval]
        max_sps_sum = max(max_sps_sum, current_sps_sum)
    return round(max_sps_sum / interval, 2)

peak_left_sps = calculate_max_sps(calculate_swings_list(df_newLeftSwing))
peak_right_sps = calculate_max_sps(calculate_swings_list(df_newRightSwing))
peak_sps = calculate_max_sps(calculate_swings_list(df_newSwing))

In [ ]:
df['_timeChangeBefore'] = df.loc[:, ['_time']].diff().fillna(0)
df['_timeChangeBefore'] = (60 * df['_timeChangeBefore']) / df['_bpm']
df['_timeChangeAfter'] = abs(df.loc[:, ['_time']].diff(periods = -1).fillna(0))
df['_timeChangeAfter'] = (60 * df['_timeChangeAfter']) / df['_bpm']

#Bombs

minReactTimeBefore = float('inf')
minReactTimeAfter = float('inf')

firstType = df.loc[0, '_type']
secondType = df.loc[1, '_type']
secondLastType = df.loc[len(df) - 2, '_type']
lastType = df.loc[len(df) - 1, '_type']

if firstType == 3 & secondType != 3:
    minReactTimeAfter = df.loc[0, '_timeChangeAfter']

if lastType == 3 & secondLastType != 3:
    if (minReactTimeBefore == float('inf')) | (minReactTimeBefore > df.loc[len(df) - 1, '_timeChangeBefore']):
        minReactTimeBefore = df.loc[len(df) - 1, '_timeChangeBefore']

for i in range(len(df) - 2):
    previousType = df.loc[i, '_type']
    thisType = df.loc[i + 1, '_type']
    nextType = df.loc[i + 2, '_type']
    if thisType == 3:
        bombException = False
        


        if (previousType == 0) | (previousType == 1):
            #Check for conditions where bomb is out of path of saber
            if (df.loc[i, '_lineLayer'] == 0) & (df.loc[i + 1, '_lineLayer'] == 2): #Note is in bottom layer and bomb is in top layer
                if (df.loc[i, '_cutDirection'] == 2) | (df.loc[i, '_cutDirection'] == 3) | (df.loc[i, '_cutDirection'] == 8): #Note direction is left, right, or dot
                    bombException = True
            elif (df.loc[i, '_type'] == 0): #Note is red
                if (df.loc[i, '_lineLayer'] == 0): #Note is bottom layer
                    if (df.loc[i, '_lineIndex'] == 0): #Note has leftmost index
                        if (df.loc[i + 1, '_lineIndex'] == 2) | (df.loc[i + 1, '_lineIndex'] == 3): #Bomb is in second rightmost or rightmost index
                            #Note direction is up, down, up left, down left, or dot
                            if (df.loc[i, '_cutDirection'] == 0) | (df.loc[i, '_cutDirection'] == 1) | (df.loc[i, '_cutDirection'] == 4) | (df.loc[i, '_cutDirection'] == 6) | (df.loc[i, '_cutDirection'] == 8):
                                bombException = True
                    elif (df.loc[i, '_lineIndex'] == 1): #Note has second leftmost index
                        if (df.loc[i + 1, '_lineIndex'] == 3): #Bomb is in rightmost index
                            if (df.loc[i, '_cutDirection'] == 0) | (df.loc[i, '_cutDirection'] == 1) | (df.loc[i, '_cutDirection'] == 8): #Note direction is up, down, or dot
                                bombException = True
                    else: #Note is in second rightmost or rightmost index
                        if (df.loc[i + 1, '_lineIndex'] == 0): #Bomb is in leftmost index
                            #Note direction is up, down, down right, or dot
                            if (df.loc[i, '_cutDirection'] == 0) | (df.loc[i, '_cutDirection'] == 1) | (df.loc[i, '_cutDirection'] == 7) | (df.loc[i, '_cutDirection'] == 8):
                                bombExcpeption = True
                elif (df.loc[i, '_lineLayer'] == 2): #Note is in top layer
                    if (df.loc[i, '_lineIndex'] == 0): #Note has leftmost index
                        if (df.loc[i + 1, '_lineIndex'] == 2) | (df.loc[i + 1, '_lineIndex'] == 3): #Bomb is in second rightmost or rightmost index
                            #Note direction is up, down, up left, down left, or dot
                            if (df.loc[i, '_cutDirection'] == 0) | (df.loc[i, '_cutDirection'] == 1) | (df.loc[i, '_cutDirection'] == 4) | (df.loc[i, '_cutDirection'] == 6) | (df.loc[i, '_cutDirection'] == 8):
                                bombException = True
                    elif (df.loc[i, '_lineIndex'] == 1): #Note has second leftmost index
                        if (df.loc[i + 1, '_lineIndex'] == 3): #Bomb is in rightmost index
                            if (df.loc[i, '_cutDirection'] == 0) | (df.loc[i, '_cutDirection'] == 1) | (df.loc[i, '_cutDirection'] == 8): #Note direction is up, down, or dot
                                bombException = True
                    else: #Note is in second rightmost or rightmost index
                        if df.loc[i + 1, '_lineIndex'] == 0: #Bomb is in leftmost index
                            if (df.loc[i, '_cutDirection'] == 0) | (df.loc[i, '_cutDirection'] == 1) | (df.loc[i, '_cutDirection'] == 8): #Note direction is up, down, or dot
                                bombException = True
            elif (df.loc[i, '_type'] == 1): #Note is blue
                if (df.loc[i, '_lineLayer'] == 0): #Note is bottom layer
                    if (df.loc[i, '_lineIndex'] == 3): #Note has rightmost index
                        if (df.loc[i + 1, '_lineIndex'] == 0) | (df.loc[i + 1, '_lineIndex'] == 1): #Bomb is in second leftmost or leftmost index
                            #Note direction is up, down, up right, down right, or dot
                            if (df.loc[i, '_cutDirection'] == 0) | (df.loc[i, '_cutDirection'] == 1) | (df.loc[i, '_cutDirection'] == 5) | (df.loc[i, '_cutDirection'] == 7) | (df.loc[i, '_cutDirection'] == 8):
                                bombException = True
                    elif (df.loc[i, '_lineIndex'] == 2): #Note has second rightmost index
                        if (df.loc[i + 1, '_lineIndex'] == 0): #Bomb is in leftmost index
                            if (df.loc[i, '_cutDirection'] == 0) | (df.loc[i, '_cutDirection'] == 1) | (df.loc[i, '_cutDirection'] == 8): #Note direction is up, down, or dot
                                bombException = True
                    else: #Note is in second leftmost or leftmost index
                        if (df.loc[i + 1, '_lineIndex'] == 3): #Bomb is in rightmost index
                            #Note direction is up, down, down left, or dot
                            if (df.loc[i, '_cutDirection'] == 0) | (df.loc[i, '_cutDirection'] == 1) | (df.loc[i, '_cutDirection'] == 6) | (df.loc[i, '_cutDirection'] == 8):
                                bombExcpeption = True
                elif (df.loc[i, '_lineLayer'] == 2): #Note is in top layer
                    if (df.loc[i, '_lineIndex'] == 3): #Note has rightmost index
                        if (df.loc[i + 1, '_lineIndex'] == 0) | (df.loc[i + 1, '_lineIndex'] == 1): #Bomb is in second leftmost or leftmost index
                            #Note direction is up, down, up right, down right, or dot
                            if (df.loc[i, '_cutDirection'] == 0) | (df.loc[i, '_cutDirection'] == 1) | (df.loc[i, '_cutDirection'] == 5) | (df.loc[i, '_cutDirection'] == 7) | (df.loc[i, '_cutDirection'] == 8):
                                bombException = True
                    elif (df.loc[i, '_lineIndex'] == 2): #Note has second rightmost index
                        if (df.loc[i + 1, '_lineIndex'] == 0): #Bomb is in leftmost index
                            if (df.loc[i, '_cutDirection'] == 0) | (df.loc[i, '_cutDirection'] == 1) | (df.loc[i, '_cutDirection'] == 8): #Note direction is up, down, or dot
                                bombException = True
                    else: #Note is in second leftmost or leftmost index
                        if df.loc[i + 1, '_lineIndex'] == 3: #Bomb is in rightmost index
                            if (df.loc[i, '_cutDirection'] == 0) | (df.loc[i, '_cutDirection'] == 1) | (df.loc[i, '_cutDirection'] == 8): #Note direction is up, down, or dot
                                bombException = True

            if (bombException == False) & ((minReactTimeBefore == float('inf')) | (minReactTimeBefore > df.loc[i + 1, '_timeChangeBefore'])):
                minReactTimeBefore = df.loc[i + 1, '_timeChangeBefore']

        if (nextType == 0) | (nextType == 1):
            #Check for conditions where bomb is out of path of saber
            if (df.loc[i + 2, '_lineLayer'] == 0) & (df.loc[i + 1, '_lineLayer'] == 2): #Note is in bottom layer and bomb is in top layer
                if (df.loc[i + 2, '_cutDirection'] == 2) | (df.loc[i + 2, '_cutDirection'] == 3) | (df.loc[i + 2, '_cutDirection'] == 8): #Note direction is left, right, or dot
                    bombException = True
            elif (df.loc[i + 2, '_type'] == 0): #Note is red
                if (df.loc[i + 2, '_lineLayer'] == 0): #Note is bottom layer
                    if (df.loc[i + 2, '_lineIndex'] == 0): #Note has leftmost index
                        if (df.loc[i + 1, '_lineIndex'] == 2) | (df.loc[i + 1, '_lineIndex'] == 3): #Bomb is in second rightmost or rightmost index
                            #Note direction is up, down, up left, down left, or dot
                            if (df.loc[i + 2, '_cutDirection'] == 0) | (df.loc[i + 2, '_cutDirection'] == 1) | (df.loc[i + 2, '_cutDirection'] == 4) | (df.loc[i + 2, '_cutDirection'] == 6) | (df.loc[i + 2, '_cutDirection'] == 8):
                                bombException = True
                    elif (df.loc[i + 2, '_lineIndex'] == 1): #Note has second leftmost index
                        if (df.loc[i + 1, '_lineIndex'] == 3): #Bomb is in rightmost index
                            if (df.loc[i + 2, '_cutDirection'] == 0) | (df.loc[i + 2, '_cutDirection'] == 1) | (df.loc[i + 2, '_cutDirection'] == 8): #Note direction is up, down, or dot
                                bombException = True
                    else: #Note is in second rightmost or rightmost index
                        if (df.loc[i + 1, '_lineIndex'] == 0): #Bomb is in leftmost index
                            #Note direction is up, down, down right, or dot
                            if (df.loc[i + 2, '_cutDirection'] == 0) | (df.loc[i + 2, '_cutDirection'] == 1) | (df.loc[i + 2, '_cutDirection'] == 7) | (df.loc[i + 2, '_cutDirection'] == 8):
                                bombException = True
                elif (df.loc[i + 2, '_lineLayer'] == 2): #Note is in top layer
                    if (df.loc[i + 2, '_lineIndex'] == 0): #Note has leftmost index
                        if (df.loc[i + 1, '_lineIndex'] == 2) | (df.loc[i + 1, '_lineIndex'] == 3): #Bomb is in second rightmost or rightmost index
                            #Note direction is up, down, up left, down left, or dot
                            if (df.loc[i + 2, '_cutDirection'] == 0) | (df.loc[i + 2, '_cutDirection'] == 1) | (df.loc[i + 2, '_cutDirection'] == 4) | (df.loc[i + 2, '_cutDirection'] == 6) | (df.loc[i + 2, '_cutDirection'] == 8):
                                bombException = True
                    elif (df.loc[i + 2, '_lineIndex'] == 1): #Note has second leftmost index
                        if (df.loc[i + 1, '_lineIndex'] == 3): #Bomb is in rightmost index
                            if (df.loc[i + 2, '_cutDirection'] == 0) | (df.loc[i + 2, '_cutDirection'] == 1) | (df.loc[i + 2, '_cutDirection'] == 8): #Note direction is up, down, or dot
                                bombException = True
                    else: #Note is in second rightmost or rightmost index
                        if df.loc[i + 1, '_lineIndex'] == 0: #Bomb is in leftmost index
                            if (df.loc[i + 2, '_cutDirection'] == 0) | (df.loc[i + 2, '_cutDirection'] == 1) | (df.loc[i + 2, '_cutDirection'] == 8): #Note direction is up, down, or dot
                                bombException = True
            elif (df.loc[i + 2, '_type'] == 1): #Note is blue
                if (df.loc[i + 2, '_lineLayer'] == 0): #Note is bottom layer
                    if (df.loc[i + 2, '_lineIndex'] == 3): #Note has rightmost index
                        if (df.loc[i + 1, '_lineIndex'] == 0) | (df.loc[i + 1, '_lineIndex'] == 1): #Bomb is in second leftmost or leftmost index
                            #Note direction is up, down, up right, down right, or dot
                            if (df.loc[i + 2, '_cutDirection'] == 0) | (df.loc[i + 2, '_cutDirection'] == 1) | (df.loc[i + 2, '_cutDirection'] == 5) | (df.loc[i + 2, '_cutDirection'] == 7) | (df.loc[i + 2, '_cutDirection'] == 8):
                                bombException = True
                    elif (df.loc[i + 2, '_lineIndex'] == 2): #Note has second rightmost index
                        if (df.loc[i + 1, '_lineIndex'] == 0): #Bomb is in leftmost index
                            if (df.loc[i + 2, '_cutDirection'] == 0) | (df.loc[i + 2, '_cutDirection'] == 1) | (df.loc[i + 2, '_cutDirection'] == 8): #Note direction is up, down, or dot
                                bombException = True
                    else: #Note is in second leftmost or leftmost index
                        if (df.loc[i + 1, '_lineIndex'] == 3): #Bomb is in rightmost index
                            #Note direction is up, down, down left, or dot
                            if (df.loc[i + 2, '_cutDirection'] == 0) | (df.loc[i + 2, '_cutDirection'] == 1) | (df.loc[i + 2, '_cutDirection'] == 6) | (df.loc[i + 2, '_cutDirection'] == 8):
                                bombExcpeption = True
                elif (df.loc[i + 2, '_lineLayer'] == 2): #Note is in top layer
                    if (df.loc[i + 2, '_lineIndex'] == 3): #Note has rightmost index
                        if (df.loc[i + 1, '_lineIndex'] == 0) | (df.loc[i + 1, '_lineIndex'] == 1): #Bomb is in second leftmost or leftmost index
                            #Note direction is up, down, up right, down right, or dot
                            if (df.loc[i + 2, '_cutDirection'] == 0) | (df.loc[i + 2, '_cutDirection'] == 1) | (df.loc[i + 2, '_cutDirection'] == 5) | (df.loc[i + 2, '_cutDirection'] == 7) | (df.loc[i + 2, '_cutDirection'] == 8):
                                bombException = True
                    elif (df.loc[i + 2, '_lineIndex'] == 2): #Note has second rightmost index
                        if (df.loc[i + 1, '_lineIndex'] == 0): #Bomb is in leftmost index
                            if (df.loc[i + 2, '_cutDirection'] == 0) | (df.loc[i + 2, '_cutDirection'] == 1) | (df.loc[i + 2, '_cutDirection'] == 8): #Note direction is up, down, or dot
                                bombException = True
                    else: #Note is in second leftmost or leftmost index
                        if df.loc[i + 1, '_lineIndex'] == 3: #Bomb is in rightmost index
                            if (df.loc[i + 2, '_cutDirection'] == 0) | (df.loc[i + 2, '_cutDirection'] == 1) | (df.loc[i + 2, '_cutDirection'] == 8): #Note direction is up, down, or dot
                                bombException = True

            if (bombException == False) & ((minReactTimeAfter == float('inf')) | (minReactTimeAfter > df.loc[i + 1, '_timeChangeAfter'])):
                minReactTimeAfter = df.loc[i + 1, '_timeChangeAfter']

In [ ]:
print('The shortest reaction time before a bomb is ' + str(math.floor(minReactTimeBefore * 1000)) + ' milliseconds.')
print('The shortest reaction time after a bomb is ' + str(math.floor(minReactTimeAfter * 1000)) + ' milliseconds.')
print('The average sps is ' + str(round(avg_sps, 2)) + ' swings per second.')
print('The njs is ' + str(njs) + '.')